In [1]:
#!import "../0-setup/Packages.ipynb"

Installed Packages Jinaga, 1.1.20 Jinaga.Notebooks, 1.1.5 Jinaga.UnitTest, 1.1.2

Loading extensions from `C:\Users\Jan\.nuget\packages\microsoft.data.analysis\0.22.0\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

# Provisioning a Supplier

A Supplier is created by a single-use principal to support two scenarios.

## Scenario 1: Manual

Jan creates the single-use principal and the Supplier.
Jan immediatly assigns administrator privileges to himself.
Jan can later add dispatchers and workers.

## Scenario 2: Self-Service

A customer logs in and creates their own single-use principal and Supplier.
The immediately assign administrator privileges to themselves.
The customer can later add dispatchers and workers through invitation links.

In [2]:
[FactType("DWS.Supplier")]
public record Supplier(User creator, Guid supplierGuid);

[FactType("DWS.Administrator")]
public record Administrator(Supplier supplier, User user, DateTime createdDate);

[FactType("DWS.Administator.Revoke")]
public record AdministratorRevoke(Administrator administrator);

Renderer.RenderTypes(typeof(Supplier), typeof(Administrator), typeof(AdministratorRevoke))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.Administrator -->
 
 DWS.Administrator 
 
 DWS.Administrator 
 
<!-- DWS.Administrator->DWS.Supplier -->
 
 DWS.Administrator->DWS.Supplier 
 
 
 supplier 
 
<!-- DWS.Administrator->Jinaga.User -->
 
 DWS.Administrator->Jinaga.User 
 
 
 user 
 
<!-- DWS.Administator.Revoke -->
 
 DWS.Administator.Revoke 
 
 DWS.Administator.Revoke 
 
<!-- DWS.Administator.Revoke->DWS.Administrator -->
 
 DWS.Administator.Revoke->DWS.Administrator 
 
 
 administrator

# Provisioning a Dispatcher

A dispatcher will log into the application and send Jan their public key.
Jan will use the public key and an operational notebook to provision the dispatcher.

In [3]:
[FactType("DWS.Dispatcher")]
public record Dispatcher(Supplier supplier, User user, DateTime createdDate);

[FactType("DWS.Dispatcher.Revoke")]
public record DispatcherRevoke(Dispatcher dispatcher);

Renderer.RenderTypes(typeof(Dispatcher), typeof(DispatcherRevoke))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Dispatcher -->
 
 DWS.Dispatcher 
 
 DWS.Dispatcher 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Dispatcher->DWS.Supplier -->
 
 DWS.Dispatcher->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Dispatcher->Jinaga.User -->
 
 DWS.Dispatcher->Jinaga.User 
 
 
 user 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.Dispatcher.Revoke -->
 
 DWS.Dispatcher.Revoke 
 
 DWS.Dispatcher.Revoke 
 
<!-- DWS.Dispatcher.Revoke->DWS.Dispatcher -->
 
 DWS.Dispatcher.Revoke->DWS.Dispatcher 
 
 
 dispatcher

# Provision a Worker

A worker will log into the mobile app and send Jan their public key.
Jan will use the public key and an operational notebook to provision the worker.

In [4]:
[FactType("DWS.Worker")]
public record Worker(Supplier supplier, User user, DateTime createdDate);

[FactType("DWS.Worker.Revoke")]
public record WorkerRevoke(Worker worker);

Renderer.RenderTypes(typeof(Worker), typeof(WorkerRevoke))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Worker -->
 
 DWS.Worker 
 
 DWS.Worker 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Worker->DWS.Supplier -->
 
 DWS.Worker->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Worker->Jinaga.User -->
 
 DWS.Worker->Jinaga.User 
 
 
 user 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.Worker.Revoke -->
 
 DWS.Worker.Revoke 
 
 DWS.Worker.Revoke 
 
<!-- DWS.Worker.Revoke->DWS.Worker -->
 
 DWS.Worker.Revoke->DWS.Worker 
 
 
 worker

In [5]:
[FactType("DWS.User.Name")]
public record UserName(User user, string value, UserName[] prior);

# Clients

A client belongs to a Supplier.
It can be deleted, and then restored.
A client has some mutable properties:
- Name

In [6]:
[FactType("DWS.Client")]
public record Client(Supplier supplier, Guid clientGuid)
{
  public Relation<ClientName> Names => Relation.Define(facts =>
    from name in facts.OfType<ClientName>()
    where name.client == this &&
      !facts.OfType<ClientName>().Any(next => next.prior.Contains(name))
    select name
  );

  public Condition IsDeleted => Condition.Define(facts =>
    facts.Any<ClientDelete>(delete => delete.client == this &&
      !facts.Any<ClientRestore>(restore => restore.clientDelete == delete)
    )
  );
}

[FactType("DWS.Client.Delete")]
public record ClientDelete(Client client, DateTime deletedDate);

[FactType("DWS.Client.Restore")]
public record ClientRestore(ClientDelete clientDelete);

[FactType("DWS.Client.Name")]
public record ClientName(Client client, string value, ClientName[] prior);

Renderer.RenderTypes(typeof(Client), typeof(ClientDelete), typeof(ClientRestore), typeof(ClientName))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Client -->
 
 DWS.Client 
 
 DWS.Client 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Client->DWS.Supplier -->
 
 DWS.Client->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.Client.Delete -->
 
 DWS.Client.Delete 
 
 DWS.Client.Delete 
 
<!-- DWS.Client.Delete->DWS.Client -->
 
 DWS.Client.Delete->DWS.Client 
 
 
 client 
 
<!-- DWS.Client.Restore -->
 
 DWS.Client.Restore 
 
 DWS.Client.Restore 
 
<!-- DWS.Client.Restore->DWS.Client.Delete -->
 
 DWS.Client.Restore->DWS.Client.Delete 
 
 
 clientDelete 
 
<!-- DWS.Client.Name -->
 
 DWS.Client.Name 
 
 DWS.Client.Name 
 
<!-- DWS.Client.Name->DWS.Client -->
 
 DWS.Client.Name->DWS.Client 
 
 
 client 
 
<!-- DWS.Client.Name->DWS.Client.Name -->
 
 DWS.Client.Name->DWS.Client.Name 
 
 
 prior

# Yard

A yard belongs to a Client.
It can be deleted, and the restored.
A yard has some mutable properties:
- Name
- Address

In [8]:
[FactType("DWS.Yard")]
public record Yard(Client client, Guid yardGuid)
{
  public Relation<YardName> Names => Relation.Define(facts =>
    from yardName in facts.OfType<YardName>()
    where yardName.yard == this &&
      !facts.Any<YardName>(next => next.prior.Contains(yardName))
    select yardName
  );

  public Relation<YardAddress> Addresses => Relation.Define(facts =>
    from yardAddress in facts.OfType<YardAddress>()
    where yardAddress.yard == this &&
      !facts.Any<YardAddress>(next => next.prior.Contains(yardAddress))
    select yardAddress
  );

  public Condition IsDeleted => Condition.Define(facts =>
    facts.Any<YardDelete>(delete => delete.yard == this &&
      !facts.Any<YardRestore>(restore => restore.yardDelete == delete)
    )
  );
}

[FactType("DWS.Yard.Delete")]
public record YardDelete(Yard yard, DateTime deletedDate);

[FactType("DWS.Yard.Restore")]
public record YardRestore(YardDelete yardDelete);

[FactType("DWS.Yard.Name")]
public record YardName(Yard yard, string value, YardName[] prior);

[FactType("DWS.Yard.Address")]
public record YardAddress(Yard yard, string street, string number, string postalCode, string place, string country, YardAddress[] prior);

Renderer.RenderTypes(typeof(Yard), typeof(YardDelete), typeof(YardRestore), typeof(YardName), typeof(YardAddress))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Yard -->
 
 DWS.Yard 
 
 DWS.Yard 
 
<!-- DWS.Client -->
 
 DWS.Client 
 
 DWS.Client 
 
<!-- DWS.Yard->DWS.Client -->
 
 DWS.Yard->DWS.Client 
 
 
 client 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Client->DWS.Supplier -->
 
 DWS.Client->DWS.Supplier 
 
 
 supplier 
 
<!-- DWS.Yard.Delete -->
 
 DWS.Yard.Delete 
 
 DWS.Yard.Delete 
 
<!-- DWS.Yard.Delete->DWS.Yard -->
 
 DWS.Yard.Delete->DWS.Yard 
 
 
 yard 
 
<!-- DWS.Yard.Restore -->
 
 DWS.Yard.Restore 
 
 DWS.Yard.Restore 
 
<!-- DWS.Yard.Restore->DWS.Yard.Delete -->
 
 DWS.Yard.Restore->DWS.Yard.Delete 
 
 
 yardDelete 
 
<!-- DWS.Yard.Name -->
 
 DWS.Yard.Name 
 
 DWS.Yard.Name 
 
<!-- DWS.Yard.Name->DWS.Yard -->
 
 DWS.Yard.Name->DWS.Yard 
 
 
 yard 
 
<!-- DWS.Yard.Name->DWS.Yard.Name -->
 
 DWS.Yard.Name->DWS.Yard.Name 
 
 
 prior 
 
<!-- DWS.Yard.Address -->
 
 DWS.Yard.Address 
 
 DWS.Yard.Address 
 
<!-- DWS.Yard.Address->DWS.Yard -->
 
 DWS.Yard.Address->DWS.Yard 
 
 
 yard 
 
<!-- DWS.Yard.Address->DWS.Yard.Address -->
 
 DWS.Yard.Address->DWS.Yard.Address 
 
 
 prior 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator

In [ ]:
[FactType("DWS.CRMClient")]
public record CRMClient(Supplier supplier, string crmId);

[FactType("DWS.CRMYard")]
public record CRMYard(CRMClient crmClient, string yardId);

[FactType("DWS.CRMYard.Address")]
public record CRMYardAddress(CRMYard crmYard, string street, string number, string city, string postalCode, string country, CRMYardAddress[] prior);

[FactType("DWS.Client.CRM")]
public record ClientCRM(Client client, CRMClient crmClient, DateTime createdDate);

[FactType("DWS.Yard.CRM")]
public record YardCRM(Yard yard, CRMYard crmYard, DateTime createdDate);

[FactType("DWS.Yard.CRM.Address")]
public record YardCRMAddress(YardAddress yardAddress, CRMYardAddress crmYardAddress, DateTime createdDate);

Renderer.RenderTypes(typeof(ClientCRM), typeof(CRMClient), typeof(YardCRM), typeof(YardCRMAddress))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Client.CRM -->
 
 DWS.Client.CRM 
 
 DWS.Client.CRM 
 
<!-- DWS.Client -->
 
 DWS.Client 
 
 DWS.Client 
 
<!-- DWS.Client.CRM->DWS.Client -->
 
 DWS.Client.CRM->DWS.Client 
 
 
 client 
 
<!-- DWS.CRMClient -->
 
 DWS.CRMClient 
 
 DWS.CRMClient 
 
<!-- DWS.Client.CRM->DWS.CRMClient -->
 
 DWS.Client.CRM->DWS.CRMClient 
 
 
 crmClient 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Client->DWS.Supplier -->
 
 DWS.Client->DWS.Supplier 
 
 
 supplier 
 
<!-- DWS.CRMClient->DWS.Supplier -->
 
 DWS.CRMClient->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.Yard.CRM -->
 
 DWS.Yard.CRM 
 
 DWS.Yard.CRM 
 
<!-- DWS.Yard -->
 
 DWS.Yard 
 
 DWS.Yard 
 
<!-- DWS.Yard.CRM->DWS.Yard -->
 
 DWS.Yard.CRM->DWS.Yard 
 
 
 yard 
 
<!-- DWS.CRMYard -->
 
 DWS.CRMYard 
 
 DWS.CRMYard 
 
<!-- DWS.Yard.CRM->DWS.CRMYard -->
 
 DWS.Yard.CRM->DWS.CRMYard 
 
 
 crmYard 
 
<!-- DWS.Yard->DWS.Client -->
 
 DWS.Yard->DWS.Client 
 
 
 client 
 
<!-- DWS.CRMYard->DWS.CRMClient -->
 
 DWS.CRMYard->DWS.CRMClient 
 
 
 crmClient 
 
<!-- DWS.Yard.CRM.Address -->
 
 DWS.Yard.CRM.Address 
 
 DWS.Yard.CRM.Address 
 
<!-- DWS.Yard.Address -->
 
 DWS.Yard.Address 
 
 DWS.Yard.Address 
 
<!-- DWS.Yard.CRM.Address->DWS.Yard.Address -->
 
 DWS.Yard.CRM.Address->DWS.Yard.Address 
 
 
 yardAddress 
 
<!-- DWS.CRMYard.Address -->
 
 DWS.CRMYard.Address 
 
 DWS.CRMYard.Address 
 
<!-- DWS.Yard.CRM.Address->DWS.CRMYard.Address -->
 
 DWS.Yard.CRM.Address->DWS.CRMYard.Address 
 
 
 crmYardAddress 
 
<!-- DWS.Yard.Address->DWS.Yard -->
 
 DWS.Yard.Address->DWS.Yard 
 
 
 yard 
 
<!-- DWS.Yard.Address->DWS.Yard.Address -->
 
 DWS.Yard.Address->DWS.Yard.Address 
 
 
 prior 
 
<!-- DWS.CRMYard.Address->DWS.CRMYard -->
 
 DWS.CRMYard.Address->DWS.CRMYard 
 
 
 crmYard 
 
<!-- DWS.CRMYard.Address->DWS.CRMYard.Address -->
 
 DWS.CRMYard.Address->DWS.CRMYard.Address 
 
 
 prior

# Tool

A tool belongs to a Supplier.
It can be deleted, and then restored.
A tool has some mutable properties:
- Name

In [9]:
[FactType("DWS.Tool")]
public record Tool(Supplier supplier, Guid toolGuid)
{
  public Relation<ToolName> Names => Relation.Define(facts =>
    from name in facts.OfType<ToolName>()
    where name.tool == this &&
      !facts.OfType<ToolName>().Any(next => next.prior.Contains(name))
    select name
  );

  public Condition IsDeleted => Condition.Define(facts =>
    facts.Any<ToolDelete>(delete => delete.tool == this &&
      !facts.Any<ToolRestore>(restore => restore.toolDelete == delete)
    )
  );
}

[FactType("DWS.Tool.Delete")]
public record ToolDelete(Tool tool, DateTime deletedDate);

[FactType("DWS.Tool.Restore")]
public record ToolRestore(ToolDelete toolDelete);

[FactType("DWS.Tool.Name")]
public record ToolName(Tool tool, string value, ToolName[] prior);

[FactType("DWS.Tool.Approved")]
public record ToolApproved(Tool tool);

Renderer.RenderTypes(typeof(Tool), typeof(ToolDelete), typeof(ToolRestore), typeof(ToolName), typeof(ToolApproved))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Tool -->
 
 DWS.Tool 
 
 DWS.Tool 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Tool->DWS.Supplier -->
 
 DWS.Tool->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.Tool.Delete -->
 
 DWS.Tool.Delete 
 
 DWS.Tool.Delete 
 
<!-- DWS.Tool.Delete->DWS.Tool -->
 
 DWS.Tool.Delete->DWS.Tool 
 
 
 tool 
 
<!-- DWS.Tool.Restore -->
 
 DWS.Tool.Restore 
 
 DWS.Tool.Restore 
 
<!-- DWS.Tool.Restore->DWS.Tool.Delete -->
 
 DWS.Tool.Restore->DWS.Tool.Delete 
 
 
 toolDelete 
 
<!-- DWS.Tool.Name -->
 
 DWS.Tool.Name 
 
 DWS.Tool.Name 
 
<!-- DWS.Tool.Name->DWS.Tool -->
 
 DWS.Tool.Name->DWS.Tool 
 
 
 tool 
 
<!-- DWS.Tool.Name->DWS.Tool.Name -->
 
 DWS.Tool.Name->DWS.Tool.Name 
 
 
 prior 
 
<!-- DWS.Tool.Approved -->
 
 DWS.Tool.Approved 
 
 DWS.Tool.Approved 
 
<!-- DWS.Tool.Approved->DWS.Tool -->
 
 DWS.Tool.Approved->DWS.Tool 
 
 
 tool

# Type of Work

Type of Work belongs to the Supplier.
It can be deleted, and then restored.
It has some mutable properties:
- Name
- Icon

In [10]:
[FactType("DWS.TypeOfWork")]
public record TypeOfWork(Supplier supplier, Guid typeOfWorkGuid)
{
  public Relation<TypeOfWorkName> Names => Relation.Define(facts =>
    from name in facts.OfType<TypeOfWorkName>()
    where name.typeOfWork == this &&
      !facts.OfType<TypeOfWorkName>().Any(next => next.prior.Contains(name))
    select name
  );

  public Relation<TypeOfWorkIcon> Icons => Relation.Define(facts =>
    from icon in facts.OfType<TypeOfWorkIcon>()
    where icon.typeOfWork == this &&
      !facts.OfType<TypeOfWorkIcon>().Any(next => next.prior.Contains(icon))
    select icon
  );

  public Condition IsDeleted => Condition.Define(facts =>
    facts.Any<TypeOfWorkDelete>(delete => delete.typeOfWork == this &&
      !facts.Any<TypeOfWorkRestore>(restore => restore.typeOfWorkDelete == delete)
    )
  );
}

[FactType("DWS.TypeOfWork.Delete")]
public record TypeOfWorkDelete(TypeOfWork typeOfWork, DateTime deletedDate);

[FactType("DWS.TypeOfWork.Restore")]
public record TypeOfWorkRestore(TypeOfWorkDelete typeOfWorkDelete);

[FactType("DWS.TypeOfWork.Name")]
public record TypeOfWorkName(TypeOfWork typeOfWork, string value, TypeOfWorkName[] prior);

[FactType("DWS.TypeOfWork.Icon")]
public record TypeOfWorkIcon(TypeOfWork typeOfWork, string hash, TypeOfWorkIcon[] prior);

Renderer.RenderTypes(typeof(TypeOfWork), typeof(TypeOfWorkDelete), typeof(TypeOfWorkRestore), typeof(TypeOfWorkName), typeof(TypeOfWorkIcon))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.TypeOfWork -->
 
 DWS.TypeOfWork 
 
 DWS.TypeOfWork 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.TypeOfWork->DWS.Supplier -->
 
 DWS.TypeOfWork->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.TypeOfWork.Delete -->
 
 DWS.TypeOfWork.Delete 
 
 DWS.TypeOfWork.Delete 
 
<!-- DWS.TypeOfWork.Delete->DWS.TypeOfWork -->
 
 DWS.TypeOfWork.Delete->DWS.TypeOfWork 
 
 
 typeOfWork 
 
<!-- DWS.TypeOfWork.Restore -->
 
 DWS.TypeOfWork.Restore 
 
 DWS.TypeOfWork.Restore 
 
<!-- DWS.TypeOfWork.Restore->DWS.TypeOfWork.Delete -->
 
 DWS.TypeOfWork.Restore->DWS.TypeOfWork.Delete 
 
 
 typeOfWorkDelete 
 
<!-- DWS.TypeOfWork.Name -->
 
 DWS.TypeOfWork.Name 
 
 DWS.TypeOfWork.Name 
 
<!-- DWS.TypeOfWork.Name->DWS.TypeOfWork -->
 
 DWS.TypeOfWork.Name->DWS.TypeOfWork 
 
 
 typeOfWork 
 
<!-- DWS.TypeOfWork.Name->DWS.TypeOfWork.Name -->
 
 DWS.TypeOfWork.Name->DWS.TypeOfWork.Name 
 
 
 prior 
 
<!-- DWS.TypeOfWork.Icon -->
 
 DWS.TypeOfWork.Icon 
 
 DWS.TypeOfWork.Icon 
 
<!-- DWS.TypeOfWork.Icon->DWS.TypeOfWork -->
 
 DWS.TypeOfWork.Icon->DWS.TypeOfWork 
 
 
 typeOfWork 
 
<!-- DWS.TypeOfWork.Icon->DWS.TypeOfWork.Icon -->
 
 DWS.TypeOfWork.Icon->DWS.TypeOfWork.Icon 
 
 
 prior

# Type of Leave

Type of Leave belongs to the Supplier.
It can be deleted, and then restored.
It has some mutable properties:
- Name
- Icon

In [11]:
[FactType("DWS.TypeOfLeave")]
public record TypeOfLeave(Supplier supplier, Guid typeOfLeaveGuid)
{
  public Relation<TypeOfLeaveName> Names => Relation.Define(facts =>
    from name in facts.OfType<TypeOfLeaveName>()
    where name.typeOfLeave == this &&
      !facts.OfType<TypeOfLeaveName>().Any(next => next.prior.Contains(name))
    select name
  );

  public Relation<TypeOfLeaveIcon> Icons => Relation.Define(facts =>
    from icon in facts.OfType<TypeOfLeaveIcon>()
    where icon.typeOfLeave == this &&
      !facts.OfType<TypeOfLeaveIcon>().Any(next => next.prior.Contains(icon))
    select icon
  );

  public Condition IsDeleted => Condition.Define(facts =>
    facts.Any<TypeOfLeaveDelete>(delete => delete.typeOfLeave == this &&
      !facts.Any<TypeOfLeaveRestore>(restore => restore.typeOfLeaveDelete == delete)
    )
  );
}

[FactType("DWS.TypeOfLeave.Delete")]
public record TypeOfLeaveDelete(TypeOfLeave typeOfLeave, DateTime deletedDate);

[FactType("DWS.TypeOfLeave.Restore")]
public record TypeOfLeaveRestore(TypeOfLeaveDelete typeOfLeaveDelete);

[FactType("DWS.TypeOfLeave.Name")]
public record TypeOfLeaveName(TypeOfLeave typeOfLeave, string value, TypeOfLeaveName[] prior);

[FactType("DWS.TypeOfLeave.Icon")]
public record TypeOfLeaveIcon(TypeOfLeave typeOfLeave, string hash, TypeOfLeaveIcon[] prior);

Renderer.RenderTypes(typeof(TypeOfLeave), typeof(TypeOfLeaveDelete), typeof(TypeOfLeaveRestore), typeof(TypeOfLeaveName), typeof(TypeOfLeaveIcon))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.TypeOfLeave -->
 
 DWS.TypeOfLeave 
 
 DWS.TypeOfLeave 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.TypeOfLeave->DWS.Supplier -->
 
 DWS.TypeOfLeave->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator 
 
<!-- DWS.TypeOfLeave.Delete -->
 
 DWS.TypeOfLeave.Delete 
 
 DWS.TypeOfLeave.Delete 
 
<!-- DWS.TypeOfLeave.Delete->DWS.TypeOfLeave -->
 
 DWS.TypeOfLeave.Delete->DWS.TypeOfLeave 
 
 
 typeOfLeave 
 
<!-- DWS.TypeOfLeave.Restore -->
 
 DWS.TypeOfLeave.Restore 
 
 DWS.TypeOfLeave.Restore 
 
<!-- DWS.TypeOfLeave.Restore->DWS.TypeOfLeave.Delete -->
 
 DWS.TypeOfLeave.Restore->DWS.TypeOfLeave.Delete 
 
 
 typeOfLeaveDelete 
 
<!-- DWS.TypeOfLeave.Name -->
 
 DWS.TypeOfLeave.Name 
 
 DWS.TypeOfLeave.Name 
 
<!-- DWS.TypeOfLeave.Name->DWS.TypeOfLeave -->
 
 DWS.TypeOfLeave.Name->DWS.TypeOfLeave 
 
 
 typeOfLeave 
 
<!-- DWS.TypeOfLeave.Name->DWS.TypeOfLeave.Name -->
 
 DWS.TypeOfLeave.Name->DWS.TypeOfLeave.Name 
 
 
 prior 
 
<!-- DWS.TypeOfLeave.Icon -->
 
 DWS.TypeOfLeave.Icon 
 
 DWS.TypeOfLeave.Icon 
 
<!-- DWS.TypeOfLeave.Icon->DWS.TypeOfLeave -->
 
 DWS.TypeOfLeave.Icon->DWS.TypeOfLeave 
 
 
 typeOfLeave 
 
<!-- DWS.TypeOfLeave.Icon->DWS.TypeOfLeave.Icon -->
 
 DWS.TypeOfLeave.Icon->DWS.TypeOfLeave.Icon 
 
 
 prior

# Tasks

A dispatcher creates a Task for a specific Client and Yard.
The task copies the address from the Yard to make it the task address.
The address can be modified.
The task has mutable properties:

- Description
- Remark
- Start date and time

The dispatcher can assign:
- Many Tools
- One Worker

In [12]:
[FactType("DWS.Task")]
public record Task(Yard yard, Guid taskGuid)
{
  public Relation<TaskClientName> ClientNames => Relation.Define(facts =>
    from name in facts.OfType<TaskClientName>()
    where name.task == this &&
      !facts.OfType<TaskClientName>().Any(next => next.prior.Contains(name))
    select name
  );

  public Relation<TaskYardName> YardNames => Relation.Define(facts =>
    from name in facts.OfType<TaskYardName>()
    where name.task == this &&
      !facts.OfType<TaskYardName>().Any(next => next.prior.Contains(name))
    select name
  );

  public Relation<TaskYardAddress> YardAddresses => Relation.Define(facts =>
    from address in facts.OfType<TaskYardAddress>()
    where address.task == this &&
      !facts.OfType<TaskYardAddress>().Any(next => next.prior.Contains(address))
    select address
  );

  public Relation<TaskToolLookup> ToolLookups => Relation.Define(facts =>
    from toolLookup in facts.OfType<TaskToolLookup>()
    where toolLookup.task == this
    select toolLookup
  );

 
  public Relation<TaskWorker> Workers => Relation.Define(facts =>
    from worker in facts.OfType<TaskWorker>()
    where worker.task == this
    select worker
  );

  public Condition IsDeleted => Condition.Define(facts =>
    facts.Any<TaskDelete>(delete => delete.task == this &&
      !facts.Any<TaskRestore>(restore => restore.taskDelete == delete)
    )
  );
}

[FactType("DWS.Task.Delete")]
public record TaskDelete(Task task, DateTime deletedDate);

[FactType("DWS.Task.Restore")]
public record TaskRestore(TaskDelete taskDelete);

[FactType("DWS.Task.ClientName")]
public record TaskClientName(Task task, string value, TaskClientName[] prior);

[FactType("DWS.Task.YardName")]
public record TaskYardName(Task task, string value, TaskYardName[] prior);

[FactType("DWS.Task.YardAddress")]
public record TaskYardAddress(Task task, string street, string number, string postalCode, string place, string country, TaskYardAddress[] prior);

[FactType("DWS.Task.Tool.Lookup")]
public record TaskToolLookup(Task task, Tool tool, DateTime createdDate);

[FactType("DWS.Task.Tool.Lookup.Delete")]
public record TaskToolLookupDelete(TaskToolLookup taskToolLookup);

[FactType("DWS.Task.Worker")]
public record TaskWorker(Task task, Worker worker, TaskWorker[] prior);

Renderer.RenderTypes(typeof(Task), typeof(TaskDelete), typeof(TaskRestore), typeof(TaskClientName), typeof(TaskYardName), typeof(TaskYardAddress), typeof(TaskToolLookup), typeof(TaskToolLookupDelete), typeof(TaskWorker), typeof(ToolName), typeof(ToolApproved))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 6.0.2 (20221011.1828)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- DWS.Task -->
 
 DWS.Task 
 
 DWS.Task 
 
<!-- DWS.Yard -->
 
 DWS.Yard 
 
 DWS.Yard 
 
<!-- DWS.Task->DWS.Yard -->
 
 DWS.Task->DWS.Yard 
 
 
 yard 
 
<!-- DWS.Client -->
 
 DWS.Client 
 
 DWS.Client 
 
<!-- DWS.Yard->DWS.Client -->
 
 DWS.Yard->DWS.Client 
 
 
 client 
 
<!-- DWS.Task.Delete -->
 
 DWS.Task.Delete 
 
 DWS.Task.Delete 
 
<!-- DWS.Task.Delete->DWS.Task -->
 
 DWS.Task.Delete->DWS.Task 
 
 
 task 
 
<!-- DWS.Task.Restore -->
 
 DWS.Task.Restore 
 
 DWS.Task.Restore 
 
<!-- DWS.Task.Restore->DWS.Task.Delete -->
 
 DWS.Task.Restore->DWS.Task.Delete 
 
 
 taskDelete 
 
<!-- DWS.Task.ClientName -->
 
 DWS.Task.ClientName 
 
 DWS.Task.ClientName 
 
<!-- DWS.Task.ClientName->DWS.Task -->
 
 DWS.Task.ClientName->DWS.Task 
 
 
 task 
 
<!-- DWS.Task.ClientName->DWS.Task.ClientName -->
 
 DWS.Task.ClientName->DWS.Task.ClientName 
 
 
 prior 
 
<!-- DWS.Task.YardName -->
 
 DWS.Task.YardName 
 
 DWS.Task.YardName 
 
<!-- DWS.Task.YardName->DWS.Task -->
 
 DWS.Task.YardName->DWS.Task 
 
 
 task 
 
<!-- DWS.Task.YardName->DWS.Task.YardName -->
 
 DWS.Task.YardName->DWS.Task.YardName 
 
 
 prior 
 
<!-- DWS.Task.YardAddress -->
 
 DWS.Task.YardAddress 
 
 DWS.Task.YardAddress 
 
<!-- DWS.Task.YardAddress->DWS.Task -->
 
 DWS.Task.YardAddress->DWS.Task 
 
 
 task 
 
<!-- DWS.Task.YardAddress->DWS.Task.YardAddress -->
 
 DWS.Task.YardAddress->DWS.Task.YardAddress 
 
 
 prior 
 
<!-- DWS.Task.Tool.Lookup -->
 
 DWS.Task.Tool.Lookup 
 
 DWS.Task.Tool.Lookup 
 
<!-- DWS.Task.Tool.Lookup->DWS.Task -->
 
 DWS.Task.Tool.Lookup->DWS.Task 
 
 
 task 
 
<!-- DWS.Tool -->
 
 DWS.Tool 
 
 DWS.Tool 
 
<!-- DWS.Task.Tool.Lookup->DWS.Tool -->
 
 DWS.Task.Tool.Lookup->DWS.Tool 
 
 
 tool 
 
<!-- DWS.Supplier -->
 
 DWS.Supplier 
 
 DWS.Supplier 
 
<!-- DWS.Tool->DWS.Supplier -->
 
 DWS.Tool->DWS.Supplier 
 
 
 supplier 
 
<!-- DWS.Task.Tool.Lookup.Delete -->
 
 DWS.Task.Tool.Lookup.Delete 
 
 DWS.Task.Tool.Lookup.Delete 
 
<!-- DWS.Task.Tool.Lookup.Delete->DWS.Task.Tool.Lookup -->
 
 DWS.Task.Tool.Lookup.Delete->DWS.Task.Tool.Lookup 
 
 
 taskToolLookup 
 
<!-- DWS.Task.Worker -->
 
 DWS.Task.Worker 
 
 DWS.Task.Worker 
 
<!-- DWS.Task.Worker->DWS.Task -->
 
 DWS.Task.Worker->DWS.Task 
 
 
 task 
 
<!-- DWS.Task.Worker->DWS.Task.Worker -->
 
 DWS.Task.Worker->DWS.Task.Worker 
 
 
 prior 
 
<!-- DWS.Worker -->
 
 DWS.Worker 
 
 DWS.Worker 
 
<!-- DWS.Task.Worker->DWS.Worker -->
 
 DWS.Task.Worker->DWS.Worker 
 
 
 worker 
 
<!-- DWS.Worker->DWS.Supplier -->
 
 DWS.Worker->DWS.Supplier 
 
 
 supplier 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- DWS.Worker->Jinaga.User -->
 
 DWS.Worker->Jinaga.User 
 
 
 user 
 
<!-- DWS.Tool.Name -->
 
 DWS.Tool.Name 
 
 DWS.Tool.Name 
 
<!-- DWS.Tool.Name->DWS.Tool -->
 
 DWS.Tool.Name->DWS.Tool 
 
 
 tool 
 
<!-- DWS.Tool.Name->DWS.Tool.Name -->
 
 DWS.Tool.Name->DWS.Tool.Name 
 
 
 prior 
 
<!-- DWS.Tool.Approved -->
 
 DWS.Tool.Approved 
 
 DWS.Tool.Approved 
 
<!-- DWS.Tool.Approved->DWS.Tool -->
 
 DWS.Tool.Approved->DWS.Tool 
 
 
 tool 
 
<!-- DWS.Client->DWS.Supplier -->
 
 DWS.Client->DWS.Supplier 
 
 
 supplier 
 
<!-- DWS.Supplier->Jinaga.User -->
 
 DWS.Supplier->Jinaga.User 
 
 
 creator